## Dataset Information

This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: 
<li>air_conditioner
<li>car_horn
<li>children_playing
<li>dog_bark
<li>drilling
<li>engine_idling
<li>gun_shot
<li>jackhammer
<li>siren
<li>street_music

## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd

## Unzip data

In [ ]:
!unzip 'drive/MyDrive/Colab Notebooks/train.zip'

## Import modules

In [ ]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import glob
import IPython.display as ipd
import random
%pylab inline

import warnings
warnings.filterwarnings('ignore')

## Loading the dataset

In [ ]:
df = pd.read_csv('Urban Sound Dataset.csv')
df.head()

In [ ]:
ipd.Audio('Train/1.wav')

## Exploratory Data Analysis

In [ ]:
data, sampling_rate = librosa.load('Train/1.wav')

In [ ]:
data

In [ ]:
sampling_rate

In [ ]:
plt.figure(figsize=(12,4))
librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
index = random.choice(df.index)

print('Class:', df['Class'][index])
data, sampling_rate = librosa.load('Train/'+str(df['ID'][index]) + '.wav')

plt.figure(figsize=(12,4))
librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
index = random.choice(df.index)

print('Class:', df['Class'][index])
data, sampling_rate = librosa.load('Train/'+str(df['ID'][index]) + '.wav')

plt.figure(figsize=(12,4))
librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
index = random.choice(df.index)

print('Class:', df['Class'][index])
data, sampling_rate = librosa.load('Train/'+str(df['ID'][index]) + '.wav')

plt.figure(figsize=(12,4))
librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,7))
sns.countplot(df['Class'])

## Input Split

In [ ]:
import os

def parser(row):
  # path of the file
  file_name = os.path.join('Train', str(row.ID) + '.wav')
  # load the audio file
  x, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  # extract features from the data
  mfccs = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=40).T, axis=0)

  feature = mfccs
  label = row.Class

  return [feature, label]

In [ ]:
data = df.apply(parser, axis=1)
data.columns = ['feature','label']

In [ ]:
data[0]

In [ ]:
# input split
X = np.array(list(zip(*data))[0])
y = np.array(list(zip(*data))[1])

## Label encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

le = LabelEncoder()
y = np_utils.to_categorical(le.fit_transform(y))

In [ ]:
y.shape

In [ ]:
y[0]

## Model Training

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

num_classes = 10

# model creation
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics='accuracy', optimizer='adam')

In [ ]:
# train the model
model.fit(X, y, batch_size=32, epochs=100, validation_split=0.25)